In [1]:
# tf.keras.Model  tf.keras.layers 构建模型
# tf.keras.losses 损失函数
# tf.keras.optimizer 优化器
# tf.keras.metrics 模型评估

import tensorflow as tf
X = tf.constant([[1.0, 2.0, 3.0],[4.0, 5.0, 6.0]])
y = tf.constant([[10.0],[20.0]])

class Linear(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense = tf.keras.layers.Dense(
            units=1,
            activation=None,
            kernel_initializer=tf.zeros_initializer(),
            bias_initializer=tf.zeros_initializer()
        )
    def call(self, input):
        output = self.dense(input)
        return output

model = Linear()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
print(model.variables)

for i in range(100):
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.reduce_mean(tf.square(y_pred-y))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads,model.variables))

print(model.variables)

[]
[<tf.Variable 'linear/dense/kernel:0' shape=(3, 1) dtype=float32, numpy=
array([[0.90326834],
       [1.2881176 ],
       [1.6729671 ]], dtype=float32)>, <tf.Variable 'linear/dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.38484946], dtype=float32)>]


In [2]:
# 手写数字 tf 实现
import numpy as np
class MNistLoader():
    def __init__(self):
        data = tf.keras.datasets.mnist
        # 加载数据
        (self.train_data, self.train_label),(self.test_data, self.test_label) = data.load_data()
        # 扩展维度，灰度图1通道 [batch_size, 28, 28, chanels=1]
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)
        self.train_label = self.train_label.astype(np.int32)
        self.test_label = self.test_label.astype(np.int32)
        # 样本个数
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]
    def get_batch(self, batch_size):
        # 从训练集里随机取出 batch_size 个样本
        idx = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[idx, :], self.train_label[idx]

In [3]:
# 自定义多层感知机模型
class MLPmodel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # 除第一维以外的维度展平
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(units=100, activation='relu')
        self.dense2 = tf.keras.layers.Dense(units=10)
    def call(self, input):
        x = self.flatten(input)
        x = self.dense1(x)
        x = self.dense2(x)
        output = tf.nn.softmax(x)
        return output

In [4]:
# CNN 模型
class myCNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=32,
            kernel_size=[5,5],
            padding='same',
            activation='relu'
        )
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2,2],strides=2)
        self.conv2 = tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=[5,5],
            padding='same',
            activation='relu'
        )
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2,2],strides=2)
        self.flatten = tf.keras.layers.Reshape(target_shape=(7*7*64,))
        self.dense1 = tf.keras.layers.Dense(units=1024,activation='relu')
        self.dense2 = tf.keras.layers.Dense(units=10)
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        outputs = tf.nn.softmax(x)
        return outputs


In [5]:
num_epochs = 5
batch_size = 50
learning_rate = 1e-4

# mymodel = MLPmodel()
mymodel = myCNN()
data_loader = MNistLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

num_batches = int(data_loader.num_train_data//batch_size * num_epochs)
for idx in range(num_batches):
    X,y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = mymodel(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch {}, loss {}".format(idx, loss.numpy()))
    grads = tape.gradient(loss, mymodel.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, mymodel.variables))

batch 0, loss 2.2848758697509766
batch 1, loss 2.2533669471740723
batch 2, loss 2.2707087993621826
batch 3, loss 2.1957805156707764
batch 4, loss 2.2161006927490234
batch 5, loss 2.2532389163970947
batch 6, loss 2.1625585556030273
batch 7, loss 2.2078046798706055
batch 8, loss 2.1527163982391357
batch 9, loss 2.086683511734009
batch 10, loss 2.138115644454956
batch 11, loss 2.018310308456421
batch 12, loss 2.019082546234131
batch 13, loss 2.0342397689819336
batch 14, loss 2.002370834350586
batch 15, loss 1.890604853630066
batch 16, loss 1.924211859703064
batch 17, loss 1.929531216621399
batch 18, loss 1.8067147731781006
batch 19, loss 1.8532944917678833
batch 20, loss 1.754205346107483
batch 21, loss 1.910902976989746
batch 22, loss 1.7295424938201904
batch 23, loss 1.6829949617385864
batch 24, loss 1.7180721759796143
batch 25, loss 1.7502083778381348
batch 26, loss 1.6091805696487427
batch 27, loss 1.5823537111282349
batch 28, loss 1.489078164100647
batch 29, loss 1.5161770582199097
b

KeyboardInterrupt: 

In [ ]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for idx in range(num_batches):
    start, end = idx*batch_size, (idx+1)*batch_size
    y_pred = mymodel.predict(data_loader.test_data[start : end])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start:end],
                                             y_pred=y_pred)
print("test 准确率：{}".format(sparse_categorical_accuracy.result()))